In [153]:
import numpy as np
from scipy.sparse import csc_matrix, coo_matrix
from scipy.sparse import kron as tensor_product
from scipy.sparse.linalg import expm as matrix_exponential

In [154]:
class QuantumGate:
    def __init__(self, name: str, scaling: float = 1.0):
        self.name = name
        self.size = len(name)
        self.__scaling__ = scaling
        self.__matrix_representation__ = self._set_matrix_()
    
    def _set_matrix_(self):
        pass
    
    def get_size(self):
        return self.size

    def get_name(self):
        return self.name
    
    def matrix(self, as_dense: bool = True):
        if as_dense:
            return np.array(self.__scaling__ * self.__matrix_representation__.todense()) 
        return self.__scaling__ * self.__matrix_representation__
    
    def time_evolution(self, time: float=1.0, as_dense: bool = True):
        if as_dense:
            return matrix_exponential(A=-1j*time*self.matrix())
        else:
            return csc_matrix(matrix_exponential(A=-1j*time*self.matrix()))
    
        
class PauliGate(QuantumGate):
    _X_ = coo_matrix(np.array([[0,1],[1,0]]   , dtype=np.complex64))
    _Y_ = coo_matrix(np.array([[0,-1j],[1j,0]], dtype=np.complex64))
    _Z_ = coo_matrix(np.array([[1,0],[0,-1]]  , dtype=np.complex64))
    _I_ = coo_matrix(np.array([[1,0],[0,1]]   , dtype=np.complex64))
    _allowed_gates_ = {'x': _X_, 'X': _X_,
                       'y': _Y_, 'Y': _Y_,
                       'z': _Z_, 'Z': _Z_,
                       'i': _I_, 'I': _I_}
    def __init__(self, name: str, scaling: float = 1.0):
        
        if len(name) == 1:
            if name not in list(self._allowed_gates_.keys()):
                raise ValueError(f'Unknown gate name provided, '
                                 f'should be one: {list(self._allowed_gates_.keys())}')
        elif len(name) > 1:
            for symbol in name:
                if symbol not in list(self._allowed_gates_.keys()):
                    raise ValueError(f'Unknown gate name provided, '
                                     f'should be one: {list(self._allowed_gates_.keys())}')
        else:
            raise ValueError(f'name does not have positive length.')
        super().__init__(name=name, scaling=scaling)
        
    def _set_matrix_(self):
        if len(self.name) == 1:
            return self._allowed_gates_[self.name]
        else:
            resulting_tensor_product = self._allowed_gates_[self.name[0]]
            for remaining_symbol in range(1,len(self.name)):
                resulting_tensor_product = tensor_product(resulting_tensor_product, self._allowed_gates_[self.name[remaining_symbol]], format=resulting_tensor_product.format)
            return resulting_tensor_product.asformat('csc')

class OperatorExpression:
    _arithmetic_operators_ = ['+','-']
    

In [161]:
gate_name = 'xyz'

obj = PauliGate(name=gate_name)

In [163]:
obj.matrix()

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+1.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [156]:
A = obj.time_evolution(as_dense=True)